In [48]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from src.data import make_dataset, data_helpers
import polyline
import pandas as pd


(strava_data, drive_morning, drive_evening) = make_dataset.load_data()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
pl = polyline.decode(strava_data['map'][1]["summary_polyline"])
pl2 = polyline.decode(strava_data['map'][5]["summary_polyline"])
pl3 = polyline.decode(strava_data['map'][0]["summary_polyline"])

In [50]:
distance, path = fastdtw(pl, pl2, dist=euclidean)

In [51]:
distance2, path2 = fastdtw(pl, pl3, dist=euclidean)

In [52]:
distance

0.011630573437018333

In [53]:
distance2

0.353006464334037

In [54]:
def get_start_latlng(mp):
    pl = mp['summary_polyline']
    if pl:
        return(polyline.decode(pl)[0])
    
def get_end_latlng(mp):
    pl = mp['summary_polyline']
    if pl:
        return(polyline.decode(pl)[-1])

In [55]:
strava_data['start_latlng'] = strava_data['map'].apply(get_start_latlng)
strava_data['end_latlng'] = strava_data['map'].apply(get_end_latlng)
strava_data[['start_lat', 'start_lng']] = strava_data['start_latlng'].apply(pd.Series)
strava_data[['end_lat', 'end_lng']] = strava_data['end_latlng'].apply(pd.Series)

In [101]:
from sklearn.cluster import DBSCAN
(morning_commutes, evening_commutes) = data_helpers.split_morning_evening(strava_data)

In [105]:
def get_polylines(df, pls):
    for (index, row) in df.iterrows():
        if (row['map']['summary_polyline']):
            pl = polyline.decode(row['map']['summary_polyline'])
            pls.append(pl)
    return pls


In [157]:
def fast_dtw_from_indices(a, b):
    pl = polyline.decode(strava_data['map'][a[0]]["summary_polyline"])
    pl2 = polyline.decode(strava_data['map'][b[0]]["summary_polyline"])
    distance, _ = fastdtw(pl, pl2)
    return distance

In [160]:
fast_dtw_from_indices([1], [0])

/home/greg/miniconda3/envs/commute_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  after removing the cwd from sys.path.


0.4274000000000413

In [108]:
pls = get_polylines(morning_commutes, [])

In [123]:
test = np.array(pls)

In [158]:
test = np.array(morning_commutes.index)
test = test.reshape(-1, 1)

In [159]:
db = DBSCAN(metric=fast_dtw_from_indices).fit(test)

TypeError: cannot do label indexing on <class 'pandas.core.indexes.numeric.Int64Index'> with these indexers [60.71739130434783] of <class 'numpy.float64'>